In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score

In [2]:
# read in data 
df = pd.read_csv('model_data.csv')
df.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,cash_in,cash_out,debit,payment,transfer
0,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,1,0
1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,1,0
2,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,1
3,181.00,181.0,0.00,21182.0,0.0,1,0,1,0,0,0
4,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,1,0


In [3]:
# split into label (isFraud) and data, drop label from data 
y = df['isFraud'].to_numpy()

df.drop(columns=['isFraud'], inplace=True)
X = df.to_numpy()

In [4]:
# split data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [5]:
# rescale x_train to prepare for model
scaler = MinMaxScaler(feature_range=(0,1))
x_train = scaler.fit_transform(x_train)

In [14]:
reg = tf.keras.regularizers.l2(0.001)

model = Sequential(
    [
        Dense(32, activation='relu', input_shape=(10,)),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ], name = 'model'
)

In [15]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [16]:
# apply class weights to balance data
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.array([0,1]), y=y_train)
class_weights

array([  0.50064581, 387.6101127 ])

In [17]:
# train model
history = model.fit(x_train, y_train, batch_size=32, epochs=11, class_weight={0:class_weights[0], 1:class_weights[1]})

Epoch 1/11
159066/159066 [==============================] - 181s 1ms/step - loss: 0.3951 - accuracy: 0.6668
Epoch 2/11
159066/159066 [==============================] - 171s 1ms/step - loss: 0.3771 - accuracy: 0.7056
Epoch 3/11
159066/159066 [==============================] - 168s 1ms/step - loss: 0.3702 - accuracy: 0.7317
Epoch 4/11
159066/159066 [==============================] - 172s 1ms/step - loss: 0.3648 - accuracy: 0.7456
Epoch 5/11
159066/159066 [==============================] - 167s 1ms/step - loss: 0.3609 - accuracy: 0.7542
Epoch 6/11
159066/159066 [==============================] - 170s 1ms/step - loss: 0.3582 - accuracy: 0.7636
Epoch 7/11
159066/159066 [==============================] - 171s 1ms/step - loss: 0.3561 - accuracy: 0.7703
Epoch 8/11
159066/159066 [==============================] - 171s 1ms/step - loss: 0.3547 - accuracy: 0.7762
Epoch 9/11
159066/159066 [==============================] - 167s 1ms/step - loss: 0.3529 - accuracy: 0.7808
Epoch 10/11
159066/159066 [=

In [18]:
pred = model.predict(x_test)

39767/39767 [==============================] - 27s 683us/step


In [19]:
# round predictions to binary classification
rounded_pred = np.round(pred)

In [20]:
accuracy = accuracy_score(y_test, rounded_pred)
accuracy

0.6640314838855692